In [42]:
import sklearn
import pandas as pd
from sklearn import linear_model
import numpy as np

In [43]:
# Load data
data = pd.read_csv('/Users/linhphung/Documents/Others/DS Projects/healthcare-dataset-stroke-data.csv')
data.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [44]:
# Drop missing values
data = data.dropna()

In [45]:
X = data.iloc[:,1:11]
X.head()
y = data['stroke']

In [46]:
X1 = pd.get_dummies(X, drop_first=True)
X1.head()

,age,hypertension,heart_disease,avg_glucose_level,bmi,gender_Male,gender_Other,ever_married_Yes,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Urban,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,67.0,0,1,228.69,36.6,True,False,True,False,True,False,False,True,True,False,False
2,80.0,0,1,105.92,32.5,True,False,True,False,True,False,False,False,False,True,False
3,49.0,0,0,171.23,34.4,False,False,True,False,True,False,False,True,False,False,True
4,79.0,1,0,174.12,24.0,False,False,True,False,False,True,False,False,False,True,False
5,81.0,0,0,186.21,29.0,True,False,True,False,True,False,False,True,True,False,False


In [47]:
# Fit logistic mode
logr = linear_model.LogisticRegression()
logr.fit(X1,y)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [61]:
logr.coef_
X1.columns

Index(['age', 'hypertension', 'heart_disease', 'avg_glucose_level', 'bmi',
       'gender_Male', 'gender_Other', 'ever_married_Yes',
       'work_type_Never_worked', 'work_type_Private',
       'work_type_Self-employed', 'work_type_children', 'Residence_type_Urban',
       'smoking_status_formerly smoked', 'smoking_status_never smoked',
       'smoking_status_smokes'],
      dtype='object')

In [62]:
# The coefficients
coeff_parameter = pd.DataFrame(logr.coef_.T, X1.columns, columns=['Coefficient'])
coeff_parameter

,Coefficient
age,0.072664
hypertension,0.510081
heart_disease,0.322110
avg_glucose_level,0.005615
bmi,-0.004988
gender_Male,-0.013179
gender_Other,-0.011248
ever_married_Yes,-0.211238
work_type_Never_worked,-0.109464
work_type_Private,-0.119991


In [53]:
# Identify boolean columns
bool_columns = X1.select_dtypes(include=bool).columns

# Convert boolean columns to integer
X1[bool_columns] = X1[bool_columns].astype(int)

In [65]:
# Run logistic model like R-code
import statsmodels.api as sm

# add constant to training data
x_const = sm.add_constant(X1)
# Train the logistic regression with the training data
log_reg_r = sm.Logit(y, x_const).fit()

         Current function value: 0.138850
         Iterations: 35


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [66]:
# Logistic regression results
log_reg_r.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 stroke   No. Observations:                 4909
Model:                          Logit   Df Residuals:                     4892
Method:                           MLE   Df Model:                           16
Date:                Tue, 19 Mar 2024   Pseudo R-squ.:                  0.2113
Time:                        21:09:39   Log-Likelihood:                -681.61
converged:                      False   LL-Null:                       -864.19
Covariance Type:            nonrobust   LLR p-value:                 7.107e-68
==================================================================================================
                                     coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const                             -8.3166      0.666    -12.495      0.000      -9.621      -7.012
age                                0.0735      0.006     11.578      0.000       0.061       0.086
hypertension                       0.5249      0.175      2.999      0.003       0.182       0.868
heart_disease                      0.3488      0.207      1.683      0.092      -0.057       0.755
avg_glucose_level                  0.0047      0.001      3.595      0.000       0.002       0.007
bmi                                0.0041      0.012      0.342      0.732      -0.019       0.027
gender_Male                       -0.0146      0.154     -0.095      0.925      -0.317       0.288
gender_Other                     -15.2670    2.8e+04     -0.001      1.000   -5.49e+04    5.48e+04
ever_married_Yes                  -0.1152      0.247     -0.466      0.641      -0.600       0.370
work_type_Never_worked            -9.0987    498.163     -0.018      0.985    -985.480     967.282
work_type_Private                  0.1608      0.223      0.720      0.472      -0.277       0.599
work_type_Self-employed           -0.2642      0.254     -1.038      0.299      -0.763       0.234
work_type_children                 0.6817      1.114      0.612      0.541      -1.502       2.865
Residence_type_Urban               0.0045      0.150      0.030      0.976      -0.289       0.298
smoking_status_formerly smoked     0.2753      0.247      1.114      0.265      -0.209       0.760
smoking_status_never smoked        0.2081      0.232      0.895      0.371      -0.247       0.664
smoking_status_smokes              0.5893      0.266      2.211      0.027       0.067       1.112
==================================================================================================
"""